In [4]:
pip install cssutils 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import xgboost as xgb
from bs4 import BeautifulSoup
import cssutils
import logging
from PIL import ImageColor
from pandas.api.types import is_numeric_dtype


## Assumptions

- Document only uses classes
- css styles only contain one attribute per line, sizes are in px

In [6]:
# Webpage string
webpage = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Detailed Border Styles</title>
    <style>
        /* Div 1: Using specific border sub-properties */
        #box-1 {
            color: #2c3e50;
            border-width: 2px;
            border-style: solid;
            border-color: #2c3e50;
        }

        /* Div 2: Using a dashed style */
        #box-2 {
            color: #e67e22;
            border-width: 4px;
            border-style: dashed;
            border-color: #e67e22;
        }

        /* Div 3 (Button): Thick "weight" with solid style */
        #button-1 {
            color: white;
            background-color: #2980b9;
            border-width: 6px;
            border-style: solid;
            border-color: #1a5276;
            cursor: pointer;
        }

        /* Div 4 (Button): Double line style */
        #button-2 {
            color: #27ae60;
            border-width: 8px;
            border-style: double;
            border-color: #27ae60;
            cursor: pointer;
        }
    </style>
</head>
<body>

    <div id="box-1">
        <h1>Heading Section</h1>
    </div>

    <div id="box-2">
        <p>This box uses a dashed border-style and a medium border-width.</p>
    </div>

    <div id="button-1">
        <strong>Primary Button</strong>
    </div>

    <div id="button-2">
        <strong>Secondary Button</strong>
    </div>

</body>
</html>
"""

In [67]:
# Get list of classes in document
soup = BeautifulSoup(webpage, 'html.parser')
all_tags = soup.find_all(True)

list_of_classes = []
for tag in all_tags:
  if tag.get('class'):
    list_of_classes.append(tag.get('class')[0])

cssutils.log.setLevel(logging.CRITICAL) # remove logs (annoying)
class_dict = dict()

for style in soup.find_all('style'):
    if style.string:
        # Parse the string extracted by BS4
        sheet = cssutils.parseString(style.string)

        # Iterate over the CSS rules
        for rule in sheet:
          if rule.type == rule.STYLE_RULE:
              selector = rule.selectorText
              styles = rule.style.cssText

              selector_styles = dict()
              for i in styles.split(';\n'):
                if 'color' in i.split(': ')[0]: # if color in name, change data to (R,G,B)
                  selector_styles[i.split(': ')[0]] = ImageColor.getrgb(str(i.split(': ')[1]))
                elif 'px' in i.split(': ')[1]: # if px in value, change data to float
                  selector_styles[i.split(': ')[0]] = float(i.split(': ')[1].split('px')[0])
                else:
                  selector_styles[i.split(': ')[0]] = i.split(': ')[1]

              class_dict[selector] = selector_styles

# Create data frame from class dictionary
df = pd.DataFrame(class_dict).T

# convert dataframe into numeric for embeddings
categorical_cols = []
for col in df:
  # Convert colour columns into categories
  if 'color' in col:
    df[col] = df[col].apply(lambda x: x if isinstance(x, tuple) else (-1,-1,-1)) # transparent
    df[[col+'_R',col+'_G',col+'_B']] = pd.DataFrame(df[col].tolist(), index=df.index)
    df = df.drop(columns=[col])
  # detect non-numeric columns as categories
  else:
    df[col] = pd.to_numeric(df[col], errors='ignore')

    if not is_numeric_dtype(df[col]):
      categorical_cols.append(col)

df=pd.get_dummies(df, columns=categorical_cols, dtype=int)

### REPLACE TRUE/FALSE with 1/0

df



/tmp/ipython-input-1569668628.py:48: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipython-input-1569668628.py:48: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


,border-width,color_R,color_G,color_B,border-color_R,border-color_G,border-color_B,background-color_R,background-color_G,background-color_B,border-style_dashed,border-style_double,border-style_solid,cursor_pointer
#box-1,2.0,44,62,80,44,62,80,-1,-1,-1,0,0,1,0
#box-2,4.0,230,126,34,230,126,34,-1,-1,-1,1,0,0,0
#button-1,6.0,255,255,255,26,82,118,41,128,185,0,0,1,1
#button-2,8.0,39,174,96,39,174,96,-1,-1,-1,0,1,0,1


In [ ]:
def vectorize_css(webpage):
  # Get list of classes in document
  soup = BeautifulSoup(webpage, 'html.parser')
  all_tags = soup.find_all(True)

  list_of_classes = []
  for tag in all_tags:
    if tag.get('class'):
      list_of_classes.append(tag.get('class')[0])

  cssutils.log.setLevel(logging.CRITICAL) # remove logs (annoying)
  class_dict = dict()

  for style in soup.find_all('style'):
      if style.string:
          # Parse the string extracted by BS4
          sheet = cssutils.parseString(style.string)

          # Iterate over the CSS rules
          for rule in sheet:
            if rule.type == rule.STYLE_RULE:
                selector = rule.selectorText
                styles = rule.style.cssText

                selector_styles = dict()
                for i in styles.split(';\n'):
                  if 'color' in i.split(': ')[0]: # if color in name, change data to (R,G,B)
                    selector_styles[i.split(': ')[0]] = ImageColor.getrgb(str(i.split(': ')[1]))
                  elif 'px' in i.split(': ')[1]: # if px in value, change data to float
                    selector_styles[i.split(': ')[0]] = float(i.split(': ')[1].split('px')[0])
                  else:
                    selector_styles[i.split(': ')[0]] = i.split(': ')[1]

                class_dict[selector] = selector_styles

  # Create data frame from class dictionary
  df = pd.DataFrame(class_dict).T

  # convert dataframe into numeric for embeddings
  categorical_cols = []
  for col in df:
    # Convert colour columns into categories
    if 'color' in col:
      df[col] = df[col].apply(lambda x: x if isinstance(x, tuple) else (-1,-1,-1)) # transparent
      df[[col+'_R',col+'_G',col+'_B']] = pd.DataFrame(df[col].tolist(), index=df.index)
      df = df.drop(columns=[col])
    # detect non-numeric columns as categories
    else:
      df[col] = pd.to_numeric(df[col], errors='ignore')

      if not is_numeric_dtype(df[col]):
        categorical_cols.append(col)

  df=pd.get_dummies(df, columns=categorical_cols, dtype=int)
  df.insert(0, "row_id", range(len(df)))
  return df

vectorize_css(webpage)



C:\Users\lawre\AppData\Local\Temp\ipykernel_15456\1171706971.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Users\lawre\AppData\Local\Temp\ipykernel_15456\1171706971.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


,row_id,border-width,color_R,color_G,color_B,border-color_R,border-color_G,border-color_B,background-color_R,background-color_G,background-color_B,border-style_dashed,border-style_double,border-style_solid,cursor_pointer
#box-1,0,2.0,44,62,80,44,62,80,-1,-1,-1,0,0,1,0
#box-2,1,4.0,230,126,34,230,126,34,-1,-1,-1,1,0,0,0
#button-1,2,6.0,255,255,255,26,82,118,41,128,185,0,0,1,1
#button-2,3,8.0,39,174,96,39,174,96,-1,-1,-1,0,1,0,1


: 